In [31]:
# Importando bibliotecas
import pandas as pd
from sqlite3.dbapi2 import connect
import sqlite3
import csv
import os

In [32]:
# criando banco de dados
conection = sqlite3.connect('super_market.db')

In [33]:
# Conectando ao banco de dados
cursor = conection.cursor()

In [49]:
# Salvando o banco de dados
conection.commit()

In [ ]:
# Desconectando o Banco de dados
conection.close()

In [42]:
# consultar nome das tabelas
result = cursor.execute("select name from sqlite_master")
result.fetchall()

[('Products',)]

In [46]:
cursor.execute("""
    CREATE TABLE Products (
        OrderID INTEGER PRIMARY KEY,
        Item TEXT NOT NULL,
        Category TEXT,
        Stock INTEGER DEFAULT 0,
        Type TEXT,
        Purchase_Price REAL,
        Sale_Price REAL CHECK(Sale_Price >= 0)
    )
""")

In [36]:
# Caminho para o CSV
csv_path = '/content/products_supermarket.csv'

with open(csv_path, newline='', encoding='utf-8') as csvfile:

    reader = csv.DictReader(csvfile)

    original_columns = reader.fieldnames
    cleaned_column_names = [col.strip().replace(" ", "_") for col in original_columns]
    escaped_column_names = [f"`{col}`" for col in cleaned_column_names]

    sql_columns = ', '.join([f"{col} TEXT" for col in escaped_column_names])
    cursor.execute(f"CREATE TABLE Products ({sql_columns})")

    for row in reader:
        cleaned_row = {col.strip().replace(" ", "_"): value for col, value in row.items()}
        placeholders = ', '.join([f":{col}" for col in cleaned_column_names])
        column_list_sql = ', '.join(escaped_column_names)

        sql_insert = f"INSERT INTO Products ({column_list_sql}) VALUES ({placeholders})"
        cursor.execute(sql_insert, cleaned_row)

In [58]:
# buscar todos os registros
cursor.execute("SELECT * FROM Products")
print(cursor.fetchall())

[(1, 'Rice', 'Basic Foods', 120, '5kg pack', 3.32, 4.38), (2, 'Beans', 'Basic Foods', 95, '1kg pack', 1.49, 2.11), (3, 'Pasta', 'Basic Foods', 150, '500g pack', 0.56, 0.97), (4, 'Soybean Oil', 'Basic Foods', 80, '900ml bottle', 1.2, 1.74), (5, 'Sugar', 'Basic Foods', 110, '1kg pack', 0.76, 1.14), (6, 'Salt', 'Basic Foods', 200, '1kg pack', 0.26, 0.53), (7, 'Wheat Flour', 'Basic Foods', 75, '1kg pack', 0.74, 1.2), (8, 'Cornmeal', 'Basic Foods', 60, '500g pack', 0.62, 1.04), (9, 'Milk', 'Dairy', 180, '1L carton', 0.72, 1.14), (10, 'Butter', 'Dairy', 65, '200g block', 1.56, 2.37), (11, 'Margarine', 'Dairy', 70, '500g block', 1.2, 1.85), (12, 'Mozzarella Cheese', 'Dairy', 45, '100g slice', 0.44, 0.79), (13, 'Yogurt', 'Dairy', 90, '170g cup', 0.32, 0.62), (14, 'Cream Cheese', 'Dairy', 55, '200g tub', 0.91, 1.56), (15, 'Heavy Cream', 'Dairy', 50, '200ml box', 0.69, 1.14), (16, 'Condensed Milk', 'Dairy', 40, '395g can', 1.14, 1.76), (17, 'French Bread', 'Bakery', 200, 'Unit', 0.06, 0.18), (18

In [59]:
cursor.execute("PRAGMA table_info(Products)")
coluns = cursor.fetchall()

# Mostra só os nomes das colunas
coluns_name = [colun[1] for colun in coluns]
print(coluns_name)

['OrderID', 'Item', 'Category', 'Stock', 'Type', 'Purchase_Price', 'Sale_Price']


In [60]:
cursor.execute("SELECT * FROM Products")

# Transforma os dados em uma lista de tuplas
products_market = cursor.fetchall()

# Conta o total de registros
total_products = len(products_market)

# Imprime o total
print("Total of item:", total_products)

Total of item: 100


In [57]:
# Data Frame
cursor.execute("SELECT * FROM Products")

# Extrai os nomes das colunas
columns = [desc[0] for desc in cursor.description]

# Busca todas as linhas da tabela
data = cursor.fetchall()

# Cria o DataFrame
df = pd.DataFrame(data, columns=columns)

# Mostra o DataFrame (tabela adaptável)
df

,OrderID,Item,Category,Stock,Type,Purchase_Price,Sale_Price
0,1,Rice,Basic Foods,120,5kg pack,3.32,4.38
1,2,Beans,Basic Foods,95,1kg pack,1.49,2.11
2,3,Pasta,Basic Foods,150,500g pack,0.56,0.97
3,4,Soybean Oil,Basic Foods,80,900ml bottle,1.20,1.74
4,5,Sugar,Basic Foods,110,1kg pack,0.76,1.14
...,...,...,...,...,...,...,...
95,96,Candles,Utensils/Disposables,25,3-unit pack,0.69,1.56
96,97,Batteries,Household Items,40,2-unit pack,1.04,2.09
97,98,Lighter/Match,Household Items,50,Unit,0.26,0.69
98,99,Dog Food,Pet,30,1kg pack,2.27,3.50


In [51]:
cursor.execute("SELECT * FROM Products")
products = cursor.fetchall()
for row in products:
    print(row)

(1, 'Rice', 'Basic Foods', 120, '5kg pack', 3.32, 4.38)
(2, 'Beans', 'Basic Foods', 95, '1kg pack', 1.49, 2.11)
(3, 'Pasta', 'Basic Foods', 150, '500g pack', 0.56, 0.97)
(4, 'Soybean Oil', 'Basic Foods', 80, '900ml bottle', 1.2, 1.74)
(5, 'Sugar', 'Basic Foods', 110, '1kg pack', 0.76, 1.14)
(6, 'Salt', 'Basic Foods', 200, '1kg pack', 0.26, 0.53)
(7, 'Wheat Flour', 'Basic Foods', 75, '1kg pack', 0.74, 1.2)
(8, 'Cornmeal', 'Basic Foods', 60, '500g pack', 0.62, 1.04)
(9, 'Milk', 'Dairy', 180, '1L carton', 0.72, 1.14)
(10, 'Butter', 'Dairy', 65, '200g block', 1.56, 2.37)
(11, 'Margarine', 'Dairy', 70, '500g block', 1.2, 1.85)
(12, 'Mozzarella Cheese', 'Dairy', 45, '100g slice', 0.44, 0.79)
(13, 'Yogurt', 'Dairy', 90, '170g cup', 0.32, 0.62)
(14, 'Cream Cheese', 'Dairy', 55, '200g tub', 0.91, 1.56)
(15, 'Heavy Cream', 'Dairy', 50, '200ml box', 0.69, 1.14)
(16, 'Condensed Milk', 'Dairy', 40, '395g can', 1.14, 1.76)
(17, 'French Bread', 'Bakery', 200, 'Unit', 0.06, 0.18)
(18, 'Sliced Bread', 

In [53]:
# Lê a tabela toda como um DataFrame
df = pd.read_sql_query("SELECT * FROM Products", conection)

# Visualiza o DataFrame
df.head()

,OrderID,Item,Category,Stock,Type,Purchase_Price,Sale_Price
0,1,Rice,Basic Foods,120,5kg pack,3.32,4.38
1,2,Beans,Basic Foods,95,1kg pack,1.49,2.11
2,3,Pasta,Basic Foods,150,500g pack,0.56,0.97
3,4,Soybean Oil,Basic Foods,80,900ml bottle,1.20,1.74
4,5,Sugar,Basic Foods,110,1kg pack,0.76,1.14
